In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
ROOT_PATH = '../..'

In [26]:
import pandas as pd
import numpy as np

dataset95 = pd.read_csv(f'{ROOT_PATH}/datasets/products_0.995.csv', low_memory=False)

In [3]:
dataset95

,code,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_en,packaging_text,brands,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,50,Financiers aux Amandes,NaN,Pâtisseries fondantes à la poudre d'amande.,660 g,"30 emballages individuels, Boîte carton","fr:30-emballages-individuels,fr:boite-carton","fr:30-emballages-individuels,fr:boite-carton",NaN,Bijou,...,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18166,Ultimate Crumpets,NaN,NaN,330 g e,"Plastique, Sachet, en:card-sleeve, en:mixed pl...","en:plastic,en:bag,en:card-sleeve,en:mixed-plas...","Plastic,Bag,Card-sleeve,Mixed-plastic-sleeve",NaN,Marks & Spencer,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000006754,Cœur coulant au chocolat - au caramel-beurre salé,NaN,fondant au chocolat et au caramel,160 g (2 x 80 g),"Plastique, Carton","en:plastic,en:cardboard","Plastic,Cardboard","1 emballage carton à recycler, emballages plas...",Bonne Maman,...,NaN,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20923301319003905,Merluza del cabo filletes,NaN,NaN,435 g,"Plástico, Buit","en:plastic,es:buit","Plastic,es:buit",NaN,BonÀrea,...,NaN,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,25751,Honey roast ham,NaN,morceaux de jambon rôti traité en salaison et ...,120 g,"Plastique, Barquette","en:plastic,en:tray","Plastic,Tray",NaN,"Marks & Spencer,M&S",...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24464,9578181777048,Allioli,NaN,Ajoaceite,190 g,Caja,en:box,Box,NaN,Chovi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24465,98147861,Wok omáčka Pangang,NaN,Omáčka na asijský způsob s bambusovými výhonky...,490 g,Glass,en:glass,Glass,GL 70,"Vitasia,Lidl",...,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24466,9900010011557,Miel de Camargue,NaN,Miel,250 g,"Verre, Bocal","en:glass,en:jar","Glass,Jar",NaN,L. Boulaire,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24467,990530101007842951,Paupiette de veau,NaN,NaN,320g,"Barquette plastique,plastique","en:plastic,fr:barquette-plastique","Plastic,fr:barquette-plastique",NaN,Agis,...,5.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
for i, c in enumerate(dataset95.columns):
  print(f'{c},', end=' ')
  if (i+1) % 8 == 0:
    print('')

code, product_name, abbreviated_product_name, generic_name, quantity, packaging, packaging_tags, packaging_en, 
packaging_text, brands, brands_tags, categories, categories_tags, categories_en, origins, origins_tags, 
origins_en, manufacturing_places, manufacturing_places_tags, labels, labels_tags, labels_en, emb_codes, emb_codes_tags, 
first_packaging_code_geo, cities, cities_tags, purchase_places, stores, countries, countries_tags, countries_en, 
ingredients_text, ingredients_tags, ingredients_analysis_tags, allergens, allergens_en, traces, traces_tags, traces_en, 
serving_size, serving_quantity, no_nutrition_data, additives_n, additives, additives_tags, additives_en, nutriscore_score, 
nutriscore_grade, nova_group, pnns_groups_1, pnns_groups_2, food_groups, food_groups_tags, food_groups_en, brand_owner, 
ecoscore_score, ecoscore_grade, nutrient_levels_tags, product_quantity, owner, data_quality_errors_tags, popularity_tags, completeness, 
main_category, main_category_en, image_url, i

### Occurences of different colums

In [ ]:
N = len(dataset95)

fr_with_nutrition = len(dataset95[dataset95['nutrition-score-fr_100g'].apply(lambda x: x.is_integer())])
print(f'FR with nutrition: {fr_with_nutrition} | {format(100*fr_with_nutrition/N, ".1f")}%')

uk_with_nutrition = len(dataset95[dataset95['nutrition-score-uk_100g'].apply(lambda x: x.is_integer())])
print(f'UK with nutrition: {uk_with_nutrition} | {format(100*uk_with_nutrition/N, ".1f")}%')

carbon_footprint = len(dataset95[dataset95['carbon-footprint-from-meat-or-fish_100g'].apply(lambda x: x.is_integer())])
print(f'Carbon footprint: {carbon_footprint} | {format(100*carbon_footprint/N, ".1f")}%')


FR with nutrition: 21524 | 88.0%
UK with nutrition: 3 | 0.0%
Carbon footprint: 580 | 2.4%


### The brands which occur the most often in the dataset

In [ ]:
# Split the 'brands' column by ',' and stack the resulting DataFrames
split_brands = dataset95['brands'].str.split(',', expand=True).stack()

# Remove leading/trailing whitespaces
split_brands = split_brands.str.strip()

# Count the occurrences of each brand
brand_counts = split_brands.value_counts()

print(brand_counts[:40])

Carrefour                                       1673
Auchan                                           851
Picard                                           527
U                                                524
Marque Repère                                    506
Casino                                           396
Lidl                                             322
Fleury Michon                                    274
Nestlé                                           264
Monoprix                                         263
Leader Price                                     243
Coop                                             238
Carrefour Bio                                    203
Danone                                           191
Sans marque                                      186
Intermarché                                      178
Savencia                                         159
Herta                                            155
Milbona                                       

### Type of packaging used

In [14]:
# count packaging used
country_tags = ['fr', 'es']
packaging = {}
for ps in dataset95['packaging_en']:
  ps = str(ps).split(',')
  for p in ps:
    if p[:2] in country_tags:
      continue
    packaging[p] = packaging.get(p, 0) + 1
    
# transform to tuple and sort
packaging = [(k, v) for k,v in packaging.items()]
packaging.sort(key=lambda x: x[1], reverse=True)

packaging[:30]

[('Plastic', 11915),
 ('Fresh', 4974),
 ('Cardboard', 4693),
 ('Tray', 3154),
 ('Bag', 1872),
 ('Metal', 1763),
 ('Protective gas', 1445),
 ('Box', 1436),
 ('Frozen', 1262),
 ('Glass', 1195),
 ('Pot', 1163),
 ('Canned', 899),
 ('Paper', 860),
 ('Film', 836),
 ('Recyclable Metals', 773),
 ('Aluminium', 768),
 ('Vacuum-packed', 752),
 ('Bottle', 720),
 ('Seal', 678),
 ('Pp-polypropylene', 628),
 ('Jar', 572),
 ('Dry', 540),
 ('Tetra Pak', 489),
 ('Sleeve', 477),
 ('Brick', 378),
 ('Container', 368),
 ('Other-plastics', 362),
 ('Can', 341),
 ('Lid', 337),
 ('Paperboard', 325)]

In [32]:
data_cities = dataset95.dropna(subset=['cities'])
data_cities # all cities values are NaN

,code,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_en,packaging_text,brands,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
